In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import random, torch
import matplotlib.pyplot as plt
import torchio as tio
import nibabel as nib


https://colab.research.google.com/github/fepegar/torchio-notebooks/blob/main/notebooks/TorchIO_tutorial.ipynb#scrollTo=-YLjzK0CP6WK

In [5]:
args = dict()
args['base_path'] = '../data/train'

In [6]:
fn_list = sorted(os.listdir(args['base_path']))
fn_list.remove('keeper')
fn = random.choice(fn_list)
print(fn)

PA000245


In [8]:
img_fn = os.path.join(args['base_path'], fn, 'image', fn + '.nii.gz')
label_fn = os.path.join(args['base_path'], fn, 'label', fn + '.nii.gz')
print(img_fn)
print(label_fn)
assert os.path.isfile(img_fn)
assert os.path.isfile(label_fn)

../data/train\PA000245\image\PA000245.nii.gz
../data/train\PA000245\label\PA000245.nii.gz


In [9]:
def read_nifti(path_to_nifti):
    return nib.load((path_to_nifti)).get_fdata()

In [10]:
img = read_nifti(img_fn)
label = read_nifti(label_fn)

In [11]:
def vis(s):
    plt.figure(figsize=(14, 10))
    plt.subplot(1,3,1)
    plt.imshow(img[:, :, s], cmap='gray')
    plt.subplot(1,3,2)
    plt.imshow(label[:, :, s], cmap='gray')
    plt.subplot(1,3,3)
    plt.imshow(img[:, :, s], cmap='gray')
    plt.imshow(label[:, :, s], alpha=0.6, cmap='Greens')
    plt.tight_layout()
    plt.show()

In [12]:
from ipywidgets import interact
import ipywidgets as widgets

interact(vis, ｓ=widgets.IntSlider(min=0, max=img.shape[-1]-1, step=1, value=75));

interactive(children=(IntSlider(value=75, description='s', max=322), Output()), _dom_classes=('widget-interact…

In [13]:
transforms =  tio.Compose([
    tio.ZNormalization(p=1),
    tio.RandomFlip(
        axes=(0, 1, 2),
        p=0,
    ),
    tio.RandomNoise(
        mean=0,
        std=(0, 0.25),
        p=0,
    ),
    tio.RandomAffine(
        scales=0.0,
        degrees=15,
        translation=5,
        isotropic=True,
        center='image' ## rotation around ccenter of image
    )
]) 

In [14]:
img_t = torch.as_tensor(img, dtype=torch.float, device='cpu').unsqueeze(dim=0)
label_t = torch.as_tensor(label, dtype=torch.float, device='cpu').unsqueeze(dim=0)
print(img_t.shape)
print(label_t.shape)

torch.Size([1, 512, 512, 323])
torch.Size([1, 512, 512, 323])


In [15]:
subject = tio.Subject(
    image=tio.ScalarImage(tensor=img_t),
    mask=tio.LabelMap(tensor=label_t)
)

In [16]:
result = transforms(subject)

In [17]:
tf_img = result['image'][tio.DATA].squeeze().detach().cpu().numpy()
tf_label = result['mask'][tio.DATA].squeeze().detach().cpu().numpy()

In [18]:
def vis_t(s):
    plt.figure(figsize=(14, 10))
    plt.subplot(1,3,1)
    plt.imshow(tf_img[:, :, s], cmap='gray')
    plt.subplot(1,3,2)
    plt.imshow(tf_label[:, :, s], cmap='gray')
    plt.subplot(1,3,3)
    plt.imshow(tf_img[:, :, s], cmap='gray')
    plt.imshow(tf_label[:, :, s], alpha=0.6, cmap='Greens')
    plt.tight_layout()
    plt.show()

In [19]:
interact(vis_t, ｓ=widgets.IntSlider(min=0, max=tf_img.shape[-1]-1, step=1, value=75));

interactive(children=(IntSlider(value=75, description='s', max=322), Output()), _dom_classes=('widget-interact…